# Name generator with pytorch

In [10]:
import pickle

In [11]:
f=  open ('./char_embedding.pkl', 'rb')
dict_char = pickle.load(f)
f.close()

In [12]:
dict_char_idx = dict()
dict_idx_char = dict()
for i, char in enumerate(dict_char.keys()):
    dict_char_idx[char] = i
for i, char in enumerate(dict_char.keys()):
    dict_idx_char[i] = char

In [13]:
import torch
from torch import nn
from torch.autograd import Variable as V
import torch.optim as optim
from torch.nn.functional import log_softmax 
import random
import torch.nn.functional as F

In [14]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.gru = nn.GRU(self.input_size, self.hidden_size,dropout = 0.5)
        self.h2o = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax() 
    def forward(self, input):
        out,_ = self.gru(input)
        out = self.h2o(out).squeeze(0).squeeze(0)
        output = self.softmax(out)
        return output
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [15]:
def predict_name_ran(char1) :
    char2=''
    char3=''
    char4='' 
    char2 = predict_char_ran10(char1)
    while char2 == '。':
        char2 = predict_char_ran10(char1)
    if char2 != '。' :
        char3 = predict_char_ran10(char2)
    if char3 != '。' :
        char4 = predict_char(char3)

    name = char1 + char2 + char3 + char4    
    return name

In [16]:
def predict_char(char):
    test = dict_char[char]
    test_tensor = torch.from_numpy(test).unsqueeze(0).unsqueeze(0).cuda()
    #print (test_tensor.shape)
    gen_test = model(test_tensor)
 
    k = int(torch.max(gen_test,0)[1])
    
    return dict_idx_char[k]

In [17]:
def predict_char_ran10(char):
    test = dict_char[char]
    test_tensor = torch.from_numpy(test).unsqueeze(0).unsqueeze(0).cuda()
    gen_test = model(test_tensor)
    k = torch.topk(gen_test,10)[1]    
    ran = int(random.random()*10)
    r= int(list(k)[ran])
    
    return dict_idx_char[r]

In [18]:
INPUT_DIM = 256
HIDDEN_DIM = 64
TARGET_DIM = len(dict_char.keys())
teacher_force = 0.87
forget_end = 0.1

model = RNN(INPUT_DIM, HIDDEN_DIM, TARGET_DIM).cuda()
model = torch.load('./dataset/model-none-max-31.pkl')

C:\Users\solit\AppData\Local\conda\conda\envs\DL\lib\site-packages\torch\serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


AttributeError: Can't get attribute '_rebuild_tensor_v2' on <module 'torch._utils' from 'C:\\Users\\solit\\AppData\\Local\\conda\\conda\\envs\\DL\\lib\\site-packages\\torch\\_utils.py'>

In [ ]:
predict_name = predict_name_ran('林')
print(predict)

# School and position recommendation with keras

In [1]:
sc_name_dict = {'CGU':"長庚大學", 'CMU':"中國醫藥大學", 'CYCU':"中原大學", 'FCU':"逢甲大學", 'KMU':"高雄醫學大學", 
                'NCCU':"政治大學", 'NCHU':"中興大學", 'NCKU':"成功大學", 'NCTU':"交通大學", 'NCUE':"彰化師範大學", 
                'NDHU':"東華大學", 'NKNU':"高雄師範大學", 'NPTU':"屏東大學", 'NSYSU':"中山大學", 'NTCU':"臺中教育大學", 
                'NTHU':"清華大學", 'NTNU':"臺灣師範大學", 'NTU':"臺灣大學", 'NTUE':"臺北教育大學", 'NUTN':"臺南大學",
                'SCU':"東吳大學", 'THU':"東海大學", 'TKU':"淡江大學", 'TNUA':"臺北藝術大學", 'UTAIPEI':"臺北市立大學",
                'YM':"陽明大學"}

In [2]:
def getCharEmbed(name):
    temp = []
    for char in name:
        try:
            temp.append(dict_char[char])
        except:
            k, v = random.choice(list(dict_char.items()))
            temp.append(v)
            
    if len(temp) == 2:
        # k, v = random.choice(list(dict_char.items()))
        user_emb = np.mean([temp[0], temp[1]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[1]))
    elif len(temp) == 3:
        user_emb = np.mean([temp[0], temp[1], temp[2]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[2]))
    elif len(temp) == 4:
        user_emb = np.mean([temp[0], temp[1], temp[2], temp[-1]], axis=0)
        con_emb = np.concatenate((temp[0], temp[1], temp[2]))
    else:
        for i in range(3):
            k, v = random.choice(list(dict_char.items()))
            temp.append(v)
        user_emb = v
        con_emb = np.concatenate((temp[0], temp[1], temp[2]))
    
    #return user_emb
    return con_emb

In [6]:
f = open ('./dataset/item_index_withEN.pkl', 'rb')
item_index_dict = pickle.load(f)
f.close()

print(item_index_dict) 

{'董事': 0, '董事長': 1, '獨立董事': 2, '副董事長': 3, '常務董事': 4, '執行業務股東': 5, 'CGU': 6, 'CMU': 7, 'CYCU': 8, 'FCU': 9, 'KMU': 10, 'NCCU': 11, 'NCHU': 12, 'NCKU': 13, 'NCTU': 14, 'NCUE': 15, 'NDHU': 16, 'NKNU': 17, 'NPTU': 18, 'NSYSU': 19, 'NTCU': 20, 'NTHU': 21, 'NTNU': 22, 'NTU': 23, 'NTUE': 24, 'NUTN': 25, 'SCU': 26, 'THU': 27, 'TKU': 28, 'TNUA': 29, 'UTAIPEI': 30, 'YM': 31}


In [7]:
def getSchool(item_index_dict, score_list):
    broad_list = []
    for i in range(len(score_list)):
        if score_list[i] == 1:
            for k, v in item_index_dict.items():
                if v == i:
                    broad_list.append(k)
    return broad_list

In [8]:
def predictSchool(name):
    count = 0
    max_score = 0
    max_sc_index = 0
    print(name)
    u = getCharEmbed(name)
    score_list = []
    ori_score_dict = {}
    for i in range(32):
        output = model_con_300.predict([[u], [i]])
        #output = model.predict([[u], [i]])
        ori_score_dict[output[0][0]] = i
        if output[0][0] > max_score:
            max_score = output[0][0]
            max_sc_index = i
        if output[0][0] > 0.82:
            score_list.append(1)
            count += 1
        else:
            score_list.append(0)
        
    ori_score_list = list(ori_score_dict.keys())
    ori_score_list.sort(reverse=True)
    
    print("你會上的機率前五高學校或職位有 :", end=" ")
    # print(ori_score_dict)
    # print(list(ori_score_dict.keys()))
    for ori in ori_score_list[:5]:
        print(list(sc_name_dict.values())[ori_score_dict[ori]-6], end=" ")
    #if count > 0:
    print("\n機率最大的是:", list(sc_name_dict.values())[max_sc_index-6])
    
    print("你有超過八成機率會上的學校或職位有 :", count, "個")
    output_list = getSchool(item_index_dict, score_list)
    print("你有超過八成機率會上的學校或職位有 :", end=" ")
    for out in output_list:
        print(sc_name_dict[out], end=" ")
    #print("\n")
    
    
        #print(max_sc_index)
    #print("\n")
    max_sc_index = 0

In [9]:
def name_generate_school(char):
    name = predict_name_ran(char)
    predictSchool(name)

In [ ]:
name_generate_school('林')